# SEQP Personalized Signal Generator
This is a Jupyter notebook designed to create a personalized version of the SEQP test signal. 

In [1]:
call = 'KD8OXT'
maidenhead = 'EN91em'

### 0) Housekeeping: Import packages and set audio parameters

In [2]:
fs = 48000;                   # sample rate, samples per second
dt = 1/fs;                    # length of a single sample, seconds per sample
wpm = 20                      # morse code words per minute

In [3]:
import morse_talk as mtalk

## 1) Generate pseudorandom signal from user's callsign and grid square.
Let's generate two seconds of static - one from the callsign, one from the grid square. We'll stick them together.

In [4]:
import numpy as np
from numpy.random import default_rng

In [5]:
rng_call = default_rng(list(call.encode('utf-8')))

In [6]:
rng_call.standard_normal(12)

array([-1.16492545, -0.20373905, -1.04660338, -0.91713151,  2.1189248 ,
       -0.59934971,  0.09140789, -1.12785716,  1.12018562,  0.81818285,
       -1.23872975,  0.08551055])

In [7]:
# ... generate the actual noise type you want ...

In [8]:
rng_grid = default_rng(list(maidenhead.encode('utf-8')))

In [9]:
rng_grid.standard_normal(12)

array([-0.81616291, -0.81423044,  0.26810335,  0.48489564, -0.48907039,
        0.88278871,  1.68193052, -0.17138467,  0.38189222,  0.2192917 ,
        0.75422993, -0.46875512])

*One consequence of this approach is that users who re-run the cell that says `second_of_static = rng...` will not get the same answer. Try something like this instead:*

In [10]:
def pseudorandom_from_string(s):
    rng = default_rng(list(s.encode('utf-8')))
    return rng.standard_normal(12) # TODO change this to match size you want

In [11]:
pseudorandom_from_string(call)

array([-1.16492545, -0.20373905, -1.04660338, -0.91713151,  2.1189248 ,
       -0.59934971,  0.09140789, -1.12785716,  1.12018562,  0.81818285,
       -1.23872975,  0.08551055])

In [12]:
pseudorandom_from_string(call)

array([-1.16492545, -0.20373905, -1.04660338, -0.91713151,  2.1189248 ,
       -0.59934971,  0.09140789, -1.12785716,  1.12018562,  0.81818285,
       -1.23872975,  0.08551055])

*... should be repeatable...*

## 2) Generate Morse signal from user's callsign. 
This signal is at 20 WPM. All callsigns must fit in the same time block... and should be received by RBN.

In [15]:
msg = 'TEST de ' + call + "   " + maidenhead+ "   "
msg = msg * 3
print(msg)

TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   TEST de KD8OXT   EN91em   


Let's convert the signal to an on-off signal:

In [24]:
msg_cw = mtalk.encode(msg, encoding_type='binary')
msg_cw

'111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111'

In [25]:
len(msg_cw)

615

We have to make sure this fits into a standard time frame, so let's prallocate an array of desired length, convert it to a list, and then add it in:

## 3) Concatenate PRN, CW, and radar chirps

Now we add in the lightsaber noises. Er, chirps. 

Chirps from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7659302.svg)](https://doi.org/10.5281/zenodo.7659302).

In [33]:
import pandas as pd
data = pd.read_csv('https://zenodo.org/record/7659302/files/seqp-test.csv?download=1')
chirps = data.to_numpy().transpose()
chirps

array([[0.        , 0.        , 0.        , ..., 0.70699947, 0.70699989,
        0.70699999]])

In [37]:
signal = [msg_cw, chirps]
signal

['111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111000000000000000111000100010101000111000000011101010001000000011101011100011101010001110111011101010001110111011100011101010111000111000000000000000100011101000111011101110111010001011101110111011100010001110111',
 array([[0.        , 0.        , 0.        , ..., 0.70699947, 0.70699989,
         0.70699999]])]

## 4) Save File
Let's save the resulting .CSV and .WAV files with mnemonic filenames that include callsign and grid square.